# Build an ML Pipeline for Airfoil noise prediction

**Airfoil**: A cross-sectional shape of a wing, blade, or sail that is designed to generate lift when air flows over it. In aeronautics, airfoils are critical components in aircraft wings, helicopter rotors, propellers, and turbine blades. The shape of an airfoil directly affects its aerodynamic performance, including lift generation, drag characteristics, and importantly, the noise it produces as air flows over its surface. Understanding and predicting airfoil noise is essential for designing quieter, more efficient aircraft and reducing environmental noise pollution.

## Setup

In [ ]:
%pip install pyspark
%pip install findspark

In [ ]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

## Part 1 - Perform ETL activity

### Import required libraries

### Create a spark session

### Load the csv file into a datadrame

### Print top 5 rows of the dataset

### Print the total number of rows in the dataset

### Drop all the duplicate rows from the dataset

### Print the total number of rows in the dataset

### Drop all the rows that contain null values from the dataset

### Print the total number of rows in the dataset

### Rename the column "SoundLevel" to "SoundLevelDecibels"

### Save the dataframe in parquet format, name the file as "NASA_airfoil_noise_cleaned.parquet"

### Part 1 - Evaluation

In [ ]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("New column name = ", df.columns[-1])

import os

print("NASA_airfoil_noise_cleaned.parquet exists :", os.path.isdir("NASA_airfoil_noise_cleaned.parquet"))